In [1]:
import sklearn
import pandas as pd
from fastai.imports import *
from fastai.structured import *

In [2]:
main_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/train.csv')
meal_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/meal_info.csv')
center_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/fulfilment_center_info.csv')

df_raw = pd.merge(main_raw, meal_raw,how='outer', on='meal_id')
df_raw = pd.merge(df_raw, center_raw, how='outer', on='center_id'); df_raw.head()

df_cat = pd.get_dummies(df_raw, columns=['meal_id', 'center_id', 'emailer_for_promotion', 'homepage_featured', 'category', 'cuisine', 'center_type'])

df_cat['week_mod4'] = df_cat.week % 4

df_cat['week_mod52'] = df_cat.week % 52

df_cat['discount'] = df_cat.base_price - df_cat.checkout_price

df_cat['discount_percent'] = df_cat.discount *100 / df_cat.base_price

df_cat.drop('id', axis=1, inplace=True)

df_cat.drop('week', axis=1, inplace=True)

df_trn, y_trn, nas = proc_df(df_cat, 'num_orders')

In [73]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

size_train = int(len(df_trn)*0.4)
x_train, x_valid = split_vals(df_trn, size_train)
y_train, y_valid = split_vals(y_trn, size_train)

In [18]:
def print_score(m):
    return metrics.mean_squared_log_error(y_valid, m.predict(x_valid))*100

In [75]:
m = ensemble.RandomForestRegressor(n_jobs=-1, max_features=0.3, min_samples_leaf=2, n_estimators=60, oob_score=False)
m.fit(x_train, y_train)
print_score(m)

42.73210453000388

In [ ]:
fi.plot('cols', 'imp', figsize=(10,6), legend=False);

In [46]:
parameters = { 'max_features':[0.08, 0.3, 0.4, 0.6, 0.8], 'min_samples_leaf':[2,3,5], 'n_estimators':[40], 'n_jobs':[-1]}

In [48]:
clf = sklearn.model_selection.GridSearchCV(m, param_grid=parameters ,
                                     scoring=print_score, n_jobs=-1,
                                    cv=5)

In [53]:
max_features = [0.08, 0.3, 0.4, 0.6, 0.8, 0.2]
min_samples_leaf = [2,3,5,1,7]

In [54]:
for f in max_features:
    for s in min_samples_leaf:
        m = ensemble.RandomForestRegressor(n_jobs=-1, max_features=f, min_samples_leaf=s, n_estimators=40, oob_score=False)
        m.fit(x_train, y_train)
        print(print_score(m), f, s)

39.988324300469344 0.08 2
40.30255459356681 0.08 3
41.641594455517 0.08 5
39.66565833693571 0.08 1
42.47558544566326 0.08 7
38.77466918906218 0.3 2
39.33108505439051 0.3 3
39.81170503272304 0.3 5
38.877247729177306 0.3 1
39.40276015889805 0.3 7
38.88351215898634 0.4 2
39.17200665492048 0.4 3
39.36374652069014 0.4 5
39.16366711915278 0.4 1
39.77086473958818 0.4 7
39.91623647127798 0.6 2
39.81612227945838 0.6 3
40.19304071268839 0.6 5
40.32774026489838 0.6 1
40.56121626432183 0.6 7
40.74683573809658 0.8 2
41.360746827643965 0.8 3
41.02439305323944 0.8 5
41.38397523326054 0.8 1
41.35711536507113 0.8 7
39.193750940711524 0.2 2
39.28515922296219 0.2 3
39.53816789767719 0.2 5
38.80647801052302 0.2 1
39.1876230710762 0.2 7


In [56]:
m = ensemble.RandomForestRegressor(n_jobs=-1, max_features=0.3, min_samples_leaf=2, n_estimators=100, oob_score=False)
m.fit(df_trn, y_trn)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.3, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [66]:
test_main_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/test_QoiMO9B.csv')

test_raw = pd.merge(test_main_raw, meal_raw,how='outer', on='meal_id')

test_raw = pd.merge(test_raw, center_raw, how='outer', on='center_id')

test_cat = pd.get_dummies(test_raw, columns=['meal_id', 'center_id', 'emailer_for_promotion', 'homepage_featured', 'category', 'cuisine', 'center_type'])

print(test_cat.shape)

test_cat['week_mod4'] = test_cat.week % 4

test_cat['week_mod52'] = test_cat.week % 52

test_cat['discount'] = test_cat.base_price - test_cat.checkout_price

test_cat['discount_percent'] = test_cat.discount *100 / test_cat.base_price

item_no = test_cat.id

test_cat.drop('id', axis=1, inplace=True)

test_cat.drop('week', axis=1, inplace=True)

preds = m.predict(test_cat)

res = pd.DataFrame(preds)

res['id'] = item_no

res['num_orders'] = preds

res.head()

res.columns

res.drop(0, axis=1, inplace=True)

res.head()

res.to_csv('/home/orange/Desktop/result.csv')

(32573, 160)


ValueError: Number of features of the model must match the input. Model n_features is 163 and input n_features is 162 

In [65]:
test_cat.shape

(32573, 164)

In [61]:
df_trn.shape

(456548, 163)

In [62]:
test_cat.shape

(32573, 163)

In [77]:
tmain_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/test_QoiMO9B.csv')
meal_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/meal_info.csv')
center_raw = pd.read_csv('/home/orange/MLHoli/Competitions/Data/fulfilment_center_info.csv')

tdf_raw = pd.merge(tmain_raw, meal_raw,how='outer', on='meal_id')
tdf_raw = pd.merge(tdf_raw, center_raw, how='outer', on='center_id')

tdf_cat = pd.get_dummies(tdf_raw, columns=['meal_id', 'center_id', 'emailer_for_promotion', 'homepage_featured', 'category', 'cuisine', 'center_type'])

tdf_cat['week_mod4'] = tdf_cat.week % 4

tdf_cat['week_mod52'] = tdf_cat.week % 52

tdf_cat['discount'] = tdf_cat.base_price - tdf_cat.checkout_price

tdf_cat['discount_percent'] = tdf_cat.discount *100 / tdf_cat.base_price

tdf_cat['month'] = tdf_cat.week / 4

tdf_cat['month'] = tdf_cat['month'].astype('int')

tdf_cat.drop('id', axis=1, inplace=True)

tdf_cat.drop('week', axis=1, inplace=True)

#tdf_trn, ty_trn, nas = proc_df(tdf_cat, 'num_orders')

preds = m.predict(tdf_cat)

res = pd.DataFrame(preds)

res['id'] = item_no

res['num_orders'] = preds

res['num_orders'] = res['num_orders'].astype('int')

res.head()

res.columns

res.drop(0, axis=1, inplace=True)

res.head()

res.to_csv('/home/orange/Desktop/result.csv')

In [70]:
tdf_cat.columns

Index(['checkout_price', 'base_price', 'city_code', 'region_code', 'op_area',
       'meal_id_1062', 'meal_id_1109', 'meal_id_1198', 'meal_id_1207',
       'meal_id_1216',
       ...
       'cuisine_Indian', 'cuisine_Italian', 'cuisine_Thai',
       'center_type_TYPE_A', 'center_type_TYPE_B', 'center_type_TYPE_C',
       'week_mod4', 'week_mod52', 'discount', 'discount_percent'],
      dtype='object', length=162)

In [71]:
df_trn.columns

Index(['checkout_price', 'base_price', 'city_code', 'region_code', 'op_area',
       'meal_id_1062', 'meal_id_1109', 'meal_id_1198', 'meal_id_1207',
       'meal_id_1216',
       ...
       'cuisine_Italian', 'cuisine_Thai', 'center_type_TYPE_A',
       'center_type_TYPE_B', 'center_type_TYPE_C', 'week_mod4', 'week_mod52',
       'month', 'discount', 'discount_percent'],
      dtype='object', length=163)